In [1]:
import tensorflow as tf
import molecule_predictionv1_1 as mp

In [2]:
dat = mp.molecule_prediction_data_wrapper()

Reading molecule structures


Done Reading molecule structures
Reading molecule prop


Done Reading molecule prop
Reading Coulomb_mat


done reading coulomb mat


In [5]:
num_epochs = 5000
batchSize = 64

batch_gen = dat.batch_gen#BatchGenerator(batch_size=batchSize)

dataset = tf.data.Dataset.\
    from_generator(batch_gen.generate, (tf.float32,tf.float32),
                   output_shapes= (tf.TensorShape([1020]), 
                                   tf.TensorShape([3])))                                                     
dataset = dataset.shuffle(buffer_size = batchSize*10) 
dataset = dataset.repeat(num_epochs).batch(batchSize)
dataset = dataset.prefetch(buffer_size = 2)
data_source =  dataset.make_one_shot_iterator()
X_batch, Y_batch = data_source.get_next()

Instructions for updating:
tf.py_func is deprecated in TF V2. Instead, use
    tf.py_function, which takes a python function which manipulates tf eager
    tensors instead of numpy arrays. It's easy to convert a tf eager tensor to
    an ndarray (just call tensor.numpy()) but having access to eager tensors
    means `tf.py_function`s can use accelerators such as GPUs as well as
    being differentiable using a gradient tape.
    


In [6]:
with tf.name_scope("dnn"):
    global X_batch, loss, prediction
    hidden1 = tf.keras.layers.Dense(2048,activation='relu')(X_batch)
    hidden2 = tf.keras.layers.Dense(2048,activation='tanh')(hidden1)
    dropout = tf.keras.layers.Dropout(rate=0.2)(hidden2)
    hidden3 =  tf.keras.layers.Dense(1024,activation='relu')(dropout)
    prediction = tf.keras.layers.Dense(3)(hidden2)
    
with tf.name_scope("loss"):
    loss = tf.losses.mean_squared_error(Y_batch, prediction)

Instructions for updating:
Colocations handled automatically by placer.


Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Instructions for updating:
Use tf.cast instead.


In [7]:
print(prediction.name)
print(X_batch.name)

dnn/dense_3/BiasAdd:0
IteratorGetNext:0


In [8]:
learning_rate = 0.001
with tf.name_scope("train"):
    global training_op
    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
    training_op = optimizer.minimize(loss) 

In [9]:
init = tf.global_variables_initializer()
saver = tf.train.Saver()

In [10]:
with tf.Session() as sess:
    init.run()
    for epoch in range(num_epochs):
        for iteration in range(batch_gen.samples// batchSize):
            _,loss_value = sess.run([training_op,loss])
            if iteration % 500 == 0:
                print("Epoch " + str(epoch) + " Step " + str(iteration) + " loss " + str(loss_value))
        if epoch % 20 == 0:
            save_path = saver.save(sess, "../models/my_model_" + str(epoch) + ".ckpt")

E:\Kaggle\Molecule\MoleculePrediction\molecule_predictionv1_1.py:123: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  result[insertHere] = array


Epoch 0 Step 0 loss 6.9171576


KeyboardInterrupt: 